## Beispiele und Anwendungen

In dieser Lektion werden wir einige Beispiele für Variationsalgorithmen und deren Anwendung untersuchen:

- So schreiben Sie einen benutzerdefinierten Variationsalgorithmus
- So wenden Sie einen Variationsalgorithmus an, um minimale Eigenwerte zu finden
- Wie man Variationsalgorithmen nutzt, um Anwendungsfälle zu lösen

### Problemdefinitionen

Stellen Sie sich vor, wir möchten einen Variationsalgorithmus verwenden, um den Eigenwert der folgenden Observablen zu ermitteln:

$$
\hat{O}_1 = 2 II - 2 XX + 3 YY - 3 ZZ,
$$

Diese Observable hat die folgenden Eigenwerte:

$$
\left\{
\begin{array}{c}
\lambda_0 = -6 \\
\lambda_1 = 4 \\
\lambda_2 = 4 \\
\lambda_3 = 6
\end{array}
\right\}
$$

Und Eigenzustände:

$$
\left\{
\begin{array}{c}
|\phi_0\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)\\
|\phi_1\rangle = \frac{1}{\sqrt{2}}(|00\rangle - |11\rangle)\\
|\phi_2\rangle = \frac{1}{\sqrt{2}}(|01\rangle - |10\rangle)\\
|\phi_3\rangle = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)
\end{array}
\right\}
$$

In [ ]:
from qiskit.quantum_info import SparsePauliOp

observable = SparsePauliOp.from_list([("II", 2), ("XX", -2), ("YY", 3), ("ZZ", -3)])

## Benutzerdefinierte VQE

Wir werden zunächst untersuchen, wie man eine VQE-Instanz manuell erstellt, um den niedrigsten Eigenwert für $\hat{O}_1$ zu finden. Dies beinhaltet eine Vielzahl von Techniken, die wir in diesem Kurs behandelt haben. 

In [ ]:
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import SparsePauliOp
from qiskit import QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator
import numpy as np

# Add your token below
service = QiskitRuntimeService(
    channel="ibm_quantum",
)

def cost_function_vqe(theta):
    observable = SparsePauliOp.from_list([("II", 2), ("XX", -2), ("YY", 3), ("ZZ", -3)])
    reference_circuit = QuantumCircuit(2)
    reference_circuit.x(0)

    variational_form = TwoLocal(
        2,
        rotation_blocks=["rz", "ry"],
        entanglement_blocks="cx",
        entanglement="linear",
        reps=1,
    )
    ansatz = reference_circuit.compose(variational_form)

    backend = service.backend("ibmq_qasm_simulator")
    
    # Use estimator to get the expected values corresponding to each ansatz
    estimator = Estimator(session=backend)
    job = estimator.run(ansatz, observable, theta)
    values = job.result().values

    return values

Mit dieser Kostenfunktion können wir optimale Parameter berechnen

In [ ]:
from qiskit.algorithms.optimizers import COBYLA

initial_theta = np.ones(8)
optimizer = COBYLA()

optimizer_result = optimizer.minimize(fun=cost_function_vqe, x0=initial_theta)

optimal_parameters = optimizer_result.x
print(optimal_parameters)

Schließlich können wir unsere optimalen Parameter verwenden, um unsere minimalen Eigenwerte zu berechnen:

In [ ]:
observable = SparsePauliOp.from_list([("II", 2), ("XX", -2), ("YY", 3), ("ZZ", -3)])
reference_circuit = QuantumCircuit(2)
reference_circuit.x(0)

variational_form = TwoLocal(
    2,
    rotation_blocks=["rz", "ry"],
    entanglement_blocks="cx",
    entanglement="linear",
    reps=1,
)
ansatz = reference_circuit.compose(variational_form)
solution = ansatz.bind_parameters(optimal_parameters)

backend = service.backend("ibmq_qasm_simulator")
estimator = Estimator(session=backend)
job = estimator.run(solution, observable)
values = job.result().values

experimental_min_eigenvalue = values[0]
print(experimental_min_eigenvalue)

In [ ]:
from numpy.linalg import eigvalsh

solution_eigenvalue = min(eigvalsh(observable.to_matrix()))
print(
    f"Percent error: {abs((experimental_min_eigenvalue - solution_eigenvalue)/solution_eigenvalue):.2e}"
)

Wie Sie sehen, kommt das Ergebnis dem Ideal sehr nahe. 

## Konfigurieren von Qiskits VQE

Der Einfachheit halber können wir auch vorhandene Qiskit- [VQE-](https://qiskit.org/documentation/stubs/qiskit.algorithms.minimum_eigensolvers.VQE.html) Implementierungen verwenden, um den niedrigsten Eigenwert der ersten Observablen $\hat{O}_1$ zu finden und uns alle Ausgabeergebnisse anzusehen.

In diesem Fall verwenden wir Folgendes:

- Wir beginnen mit der Untersuchung des Referenzoperators $\equiv I$, um die Geschwindigkeitssteigerungen zu demonstrieren, die dadurch möglich sind
- Qiskit Terras [`Estimator`](https://qiskit.org/documentation/stubs/qiskit.primitives.Estimator.html#qiskit.primitives.Estimator)
- [`SLSQP`](https://qiskit.org/documentation/stubs/qiskit.algorithms.optimizers.SLSQP.html) Optimierer (d. h. Sequential Least SQuares Programming).
- Zusätzlich werden wir den Anfangspunkt für den `SLSQP` Optimierer auf $\vec\theta_0 = (1, \cdots, 1)$ setzen.

In [ ]:
from qiskit.primitives import Estimator
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms.minimum_eigensolvers import VQE
import numpy as np

estimator = Estimator()
optimizer = SLSQP()
ansatz = TwoLocal(
    2,
    rotation_blocks=["rz", "ry"],
    entanglement_blocks="cx",
    entanglement="linear",
    reps=1,
)

vqe = VQE(estimator, ansatz, optimizer, initial_point=np.ones(8))

Nachdem wir nun die `VQE` Instanz initialisiert haben, können wir die Ergebnisse mit der Methode [`VQE.compute_minimum_eigenvalue`](https://qiskit.org/documentation/stubs/qiskit.algorithms.minimum_eigensolvers.VQE.compute_minimum_eigenvalue.html#qiskit.algorithms.minimum_eigensolvers.VQE.compute_minimum_eigenvalue) abrufen. Schauen wir uns die Ergebnisse an.

In [ ]:
result = vqe.compute_minimum_eigenvalue(observable)
print(result)

Schauen wir uns das `optimizer_result` an:

In [ ]:
print(result.optimizer_result)

Von all diesen Informationen ist jedoch der Eigenwert der wichtigste Teil. Vergleichen wir es mit dem theoretischen Wert:

In [ ]:
from numpy.linalg import eigvalsh

eigenvalues = eigvalsh(observable.to_matrix())
min_eigenvalue = eigenvalues[0]

print("EIGENVALUES:")
print(f"  - Theoretical: {min_eigenvalue}.")
print(f"  - VQE: {result.eigenvalue}")
print(
    f"Percent error >> {abs((result.eigenvalue - min_eigenvalue)/min_eigenvalue):.2e}"
)

Wie Sie sehen, kommt das Ergebnis dem Ideal sehr nahe.

Allerdings haben wir uns die Eigenzustände noch nicht angesehen, da sie nicht Teil der `results` waren. Zu diesem Zweck binden wir die optimalen Parameterwerte `result.optimal_parameters` an `results.optimal_circuit` und definieren einen [`Statevector`](https://qiskit.org/documentation/stubs/qiskit.quantum_info.Statevector.html) aus dieser gebundenen (dh nicht parametrisierten) Schaltung.

In [ ]:
from qiskit.quantum_info import Statevector

optimal_circuit = result.optimal_circuit.bind_parameters(result.optimal_parameters)
optimal_vector = Statevector(optimal_circuit)

rounded_optimal_vector = np.round(optimal_vector.data, 3)
print(f"EIGENSTATE: {rounded_optimal_vector}")

Dieses Ergebnis scheint dem theoretischen von $\frac{1}{\sqrt{2}}(|00\rangle + |11\rangle) \equiv [\frac{1}{\sqrt{2} },0,0,\frac{1}{\sqrt{2}}]$. Beachten Sie jedoch, dass Eigenvektoren bis zu einem konstanten Faktor definiert sind. Darüber hinaus sind Quantenzustände bis zu einer globalen Phase immer normalisiert und äquivalent, sodass wir leicht überprüfen können, dass diese beiden Zustandsvektoren äquivalent sind.

In [ ]:
from numpy.linalg import eigh

_, eigenvectors = eigh(observable.to_matrix())
min_eigenvector = eigenvectors.T[0]  # Note: transpose to extract by index

optimal_vector.equiv(min_eigenvector, atol=1e-4)

Wir können daraus schließen, dass der von uns erhaltene Zustand dem Idealzustand bis zu 10^{-4}$ entspricht.

### Referenzstatus hinzufügen

Im vorherigen Beispiel haben wir keinen Referenzoperator $U_R$ verwendet. Denken wir nun darüber nach, wie der ideale Eigenzustand $\frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$ erhalten werden kann. Betrachten Sie die folgende Schaltung.

In [ ]:
from qiskit import QuantumCircuit

ideal_qc = QuantumCircuit(2)
ideal_qc.h(0)
ideal_qc.cx(0, 1)

ideal_qc.draw("mpl")

Wir können schnell überprüfen, ob diese Schaltung uns den gewünschten Zustand liefert.

In [ ]:
Statevector(ideal_qc)

Nachdem wir nun gesehen haben, wie eine Schaltung aussieht, die den Lösungszustand vorbereitet, erscheint es sinnvoll, ein Hadamard-Gatter als Referenzschaltung zu verwenden, sodass der vollständige Ansatz zu Folgendem wird:

In [ ]:
reference = QuantumCircuit(2)
reference.h(0)
# Include barrier to separate reference from variational form
reference.barrier()

ref_ansatz = ansatz.decompose().compose(reference, front=True)

ref_ansatz.draw("mpl")

Für diese neue Schaltung konnte die ideale Lösung erreicht werden, wenn alle Parameter auf $0$ eingestellt waren, was bestätigt, dass die Wahl der Referenzschaltung sinnvoll ist.

Vergleichen wir nun die Anzahl der Kostenfunktionsauswertungen, Optimierungsiterationen und die benötigte Zeit mit denen des vorherigen Versuchs.

In [ ]:
num_evaluations = result.cost_function_evals
num_iterations = result.optimizer_result.nit
time = result.optimizer_time

print("NO REFERENCE STATE:")
print(f"  - Number of evaluations: {num_evaluations}")
print(f"  - Number of iterations: {num_iterations}")
print(f"  - Time: {time:.5f} seconds")

In [ ]:
# You can change the ansatz of the already defined vqe object instead of creating a new one
vqe.ansatz = ref_ansatz

ref_result = vqe.compute_minimum_eigenvalue(observable)

In [ ]:
num_evaluations_ref = ref_result.cost_function_evals
num_iterations_ref = ref_result.optimizer_result.nit
time_ref = ref_result.optimizer_time

print("ADDED REFERENCE STATE:")
print(f"  - Number of evaluations: {num_evaluations_ref}")
print(f"  - Number of iterations: {num_iterations_ref}")
print(f"  - Time: {time_ref:.5f} seconds")
print()

if num_evaluations_ref < num_evaluations:
    print(">> Number of cost function evaluations improved")
elif num_evaluations_ref > num_evaluations:
    print(">> Number of cost function evaluations worsened")
if num_iterations_ref < num_iterations:
    print(">> Number of iterations improved")
elif num_iterations_ref > num_iterations:
    print(">> Number of iterations worsened")
if time_ref < time:
    print(">> Time improved")
elif time_ref > time:
    print(">> Time worsened")

### Anfangspunkt ändern

Nachdem wir nun die Auswirkung des Hinzufügens des Referenzzustands gesehen haben, gehen wir darauf ein, was passiert, wenn wir verschiedene Anfangspunkte $\vec{\theta_0}$ wählen. Insbesondere werden wir $\vec{\theta_0}=(0,0,0,0,6,0,0,0)$ und $\vec{\theta_0}=(6,6,6,6,6) verwenden ,6,6,6,6)$. Denken Sie daran, dass, wie bei der Einführung des Referenzzustands besprochen, die ideale Lösung gefunden würde, wenn alle Parameter $0$ wären, sodass der erste Anfangspunkt weniger Auswertungen, Iterationen und Zeit erfordern sollte.

In [ ]:
vqe.initial_point = [0, 0, 0, 0, 6, 0, 0, 0]

result = vqe.compute_minimum_eigenvalue(observable)

num_evaluations = result.cost_function_evals
num_iterations = result.optimizer_result.nit
time = result.optimizer_time

print(f"INITIAL POINT: {vqe.initial_point}")
print(f"  - Number of evaluations: {num_evaluations}")
print(f"  - Number of iterations: {num_iterations}")
print(f"  - Time: {time:.5f} seconds")

In [ ]:
vqe.initial_point = 6 * np.ones(8)

result = vqe.compute_minimum_eigenvalue(observable)

num_evaluations = result.cost_function_evals
num_iterations = result.optimizer_result.nit
time = result.optimizer_time

print(f"INITIAL POINT: {vqe.initial_point}")
print(f"  - Number of evaluations: {num_evaluations}")
print(f"  - Number of iterations: {num_iterations}")
print(f"  - Time: {time:.5f} seconds")

## VQD-Beispiel

Anstatt nur nach dem niedrigsten Eigenwert unserer Observablen zu suchen, suchen wir nun nach allen $4$. Nach der Notation aus dem vorherigen Kapitel (sowie der aus der [VQD-](https://qiskit.org/documentation/stubs/qiskit.algorithms.eigensolvers.VQD.html) Klasse von Qiskit) bedeutet das $k=4$.

Denken Sie daran, dass die Kostenfunktionen von VQD sind:

$$
C_{l}(\vec{\theta}) := 
\langle \psi(\vec{\theta}) | \hat{H} | \psi(\vec{\theta})\rangle + 
\sum_{j=0}^{l-1}\beta_j |\langle \psi(\vec{\theta})| \psi(\vec{\theta^j})\rangle  |^2 
\quad \forall l\in\{1,\cdots,k\}=\{1,\cdots,4\}
$$

Dies ist besonders wichtig, da ein Vektor $\vec{\beta}=(\beta_0,\cdots,\beta_{k-1})$ (in diesem Fall $(\beta_0, \beta_1, \beta_2, \beta_3)$ ) muss als Argument übergeben werden, wenn wir das `VQD` Objekt definieren.

Außerdem werden in Qiskits Implementierung von VQD anstelle der im vorherigen Notizbuch beschriebenen effektiven Observablen die Genauigkeiten $|\langle \psi(\vec{\theta})| berücksichtigt \psi(\vec{\theta^j})\rangle |^2$ werden direkt über den [`ComputeUncompute`](https://qiskit.org/documentation/stubs/qiskit.algorithms.state_fidelities.ComputeUncompute.html) Algorithmus berechnet, der ein `Sampler` Grundelement nutzt, um die Wahrscheinlichkeit abzutasten, $|0\rangle$ für die Schaltung $U_A^\ zu erhalten Dolch(\vec{\theta})U_A(\vec{\theta^j})$. Das funktioniert genau deshalb, weil diese Wahrscheinlichkeit ist

$$
\begin{aligned}

p_0

& = |\langle 0|U_A^\dagger(\vec{\theta})U_A(\vec{\theta^j})|0\rangle|^2 \\[1mm]

& = |\big(\langle 0|U_A^\dagger(\vec{\theta})\big)\big(U_A(\vec{\theta^j})|0\rangle\big)|^2 \\[1mm]

& = |\langle \psi(\vec{\theta}) |\psi(\vec{\theta^j}) \rangle|^2 \\[1mm]

\end{aligned}
$$

Um schließlich einen neuen Optimierer auszuprobieren, verwenden wir [`COBYLA`](https://qiskit.org/documentation/stubs/qiskit.algorithms.optimizers.COBYLA.html) anstelle von `SLSQP` .

In [ ]:
from qiskit.primitives import Sampler
from qiskit.algorithms.optimizers import COBYLA
from qiskit.algorithms.state_fidelities import ComputeUncompute
from qiskit.algorithms.eigensolvers import VQD

optimizer = COBYLA()
sampler = Sampler()
estimator = Estimator()
fidelity = ComputeUncompute(sampler)

k = 4
betas = [40, 60, 30, 30]

vqd = VQD(estimator, fidelity, ansatz, optimizer, k=k, betas=betas)

Beachten Sie, dass der einzige API-Unterschied zu den vorherigen Beispielen darin besteht, dass wir statt der Methode `VQE.compute_minimum_eigenvalue` [`VQD.compute_eigenvalues`](https://qiskit.org/documentation/stubs/qiskit.algorithms.eigensolvers.VQD.compute_eigenvalues.html) ​​aufrufen. Beginnen wir mit der Untersuchung der folgenden Beobachtungsgröße:

$$
\hat{O}_2 := 2 II - 3 XX + 2 YY - 4 ZZ
$$

Diese Observable hat die folgenden Eigenwerte:

$$
\left\{
\begin{array}{c}
\lambda_0 = -7 \\
\lambda_1 = 3\\
\lambda_2 = 5 \\
\lambda_3 = 7
\end{array}
\right\}
$$

Und Eigenzustände:

$$
\left\{
\begin{array}{c}
|\phi_0\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)\\
|\phi_1\rangle = \frac{1}{\sqrt{2}}(|00\rangle - |11\rangle)\\
|\phi_2\rangle = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)\\
|\phi_3\rangle = \frac{1}{\sqrt{2}}(|01\rangle - |10\rangle)
\end{array}
\right\}
$$

In [ ]:
observable_2 = SparsePauliOp.from_list([("II", 2), ("XX", -3), ("YY", 2), ("ZZ", -4)])

result = vqd.compute_eigenvalues(observable_2)
print(result)

Das [`VQDResult`](https://qiskit.org/documentation/stubs/qiskit.algorithms.eigensolvers.VQDResult.html) das wir erhalten, ist völlig analog zum `VQEResult` . Der einzige Unterschied besteht darin, dass jedes Attribut eine Liste ist, deren $i$-tes Element dem $i$-ten Eigenwert entspricht.

Nachdem wir nun die Eigenwerte gesehen haben, vergleichen wir die experimentellen Eigenvektoren mit den theoretischen:

In [ ]:
optimal_circuits = [
    circuit.bind_parameters(parameters)
    for circuit, parameters in zip(result.optimal_circuits, result.optimal_parameters)
]
eigenstates = [Statevector(c) for c in optimal_circuits]

for i, (eigenvalue, eigenstate) in enumerate(zip(result.eigenvalues, eigenstates)):
    eigenvalue = eigenvalue.real
    eigenstate = np.round(eigenstate.data, 3).tolist()
    print(f"RESULT {i}:")
    print(f"  - {eigenvalue = :.3f}")
    print(f"  - {eigenstate = }")

Diese Ergebnisse stimmen mit den erwarteten überein, mit Ausnahme eines kleinen Näherungsfehlers und einer globalen Phase.

Lösen wir nun dieses Problem für die erste Observable $\hat{O}_1 := 2 II - 2 XX + 3 YY - 3 ZZ$.

In [ ]:
result = vqd.compute_eigenvalues(observable)

optimal_circuits = [
    circuit.bind_parameters(parameters)
    for circuit, parameters in zip(result.optimal_circuits, result.optimal_parameters)
]
eigenstates = [Statevector(c) for c in optimal_circuits]

for i, (eigenvalue, eigenstate) in enumerate(zip(result.eigenvalues, eigenstates)):
    eigenvalue = eigenvalue.real
    eigenstate = np.round(eigenstate.data, 3).tolist()
    print(f"RESULT {i}:")
    print(f"  - {eigenvalue = :.3f}")
    print(f"  - {eigenstate = }")

Hier sind die Eigenzustände, die $\lambda_1 = \lambda_2 = 4$ entsprechen, nicht die gleichen wie die erwarteten:

$$
|\phi_1\rangle = \frac{1}{\sqrt{2}}(|00\rangle - |11\rangle) \\
|\phi_2\rangle = \frac{1}{\sqrt{2}}(|01\rangle + |10\rangle)
$$

Dies geschieht genau deshalb, weil als $\lambda_1=\lambda_2$ jede komplexe Linearkombination auch ein Eigenzustand mit demselben Eigenwert ist:

$$
\begin{aligned}
\alpha_1 |\phi_1\rangle + \alpha_2 |\phi_2\rangle
& = \frac{1}{\sqrt{2}}(\alpha_1 |00\rangle + \alpha_2 |01\rangle - \alpha_2 |10\rangle - \alpha_1 |11\rangle) \\[1mm]
& \equiv \frac{1}{\sqrt{2}}[\alpha_1, \alpha_2, -\alpha_2, -\alpha_1]
\end{aligned}
$$

Genau das sehen wir mit diesen Ergebnissen.

### Betas ändern

Wie in der Instanzlektion erwähnt, sollten die Werte von $\vec{\beta}$ größer sein als die Differenz zwischen Eigenwerten. Sehen wir uns an, was passiert, wenn sie diese Bedingung mit $\hat{O}_2$ nicht erfüllen

$$
\hat{O}_2 = 2 II - 3 XX + 2 YY - 4 ZZ
$$

mit Eigenwerten

$$
\left\{
\begin{array}{c}
\lambda_0 = -7 \\
\lambda_1 = 3\\
\lambda_2 = 5 \\
\lambda_3 = 7
\end{array}
\right\}
$$

In [ ]:
vqd.betas = np.ones(4)
result = vqd.compute_eigenvalues(observable_2)

optimal_circuits = [
    circuit.bind_parameters(parameters)
    for circuit, parameters in zip(result.optimal_circuits, result.optimal_parameters)
]
eigenstates = [Statevector(c) for c in optimal_circuits]

for i, (eigenvalue, eigenstate) in enumerate(zip(result.eigenvalues, eigenstates)):
    eigenvalue = eigenvalue.real
    eigenstate = np.round(eigenstate.data, 3).tolist()
    print(f"RESULT {i}:")
    print(f"  - {eigenvalue = :.3f}")
    print(f"  - {eigenstate = }")

Diesmal gibt der Optimierer den gleichen Zustand $|\phi_0\rangle = \frac{1}{\sqrt{2}}(|00\rangle + |11\rangle)$ als vorgeschlagene Lösung für alle Eigenzustände zurück: nämlich eindeutig falsch. Dies liegt daran, dass die Betas zu klein waren, um den minimalen Eigenzustand in den aufeinanderfolgenden Kostenfunktionen zu benachteiligen. Daher wurde es in späteren Iterationen des Algorithmus nicht aus dem effektiven Suchraum ausgeschlossen und immer als bestmögliche Lösung ausgewählt.

Wir empfehlen, mit den Werten von $\vec{\beta}$ zu experimentieren und sicherzustellen, dass sie größer sind als die Differenz zwischen den Eigenwerten. 

## Quantenchemie: Grundzustand und angeregte Energielöser

Unser Ziel ist es, den Erwartungswert der Observablen zu minimieren, die die Energie darstellen (Hamiltonian $\hat{\mathcal{H}}$):

$$
\min_{\vec\theta} \langle\psi(\vec\theta)|\hat{\mathcal{H}}|\psi(\vec\theta)\rangle
$$

In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.circuit.library import HartreeFock, UCC
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.quantum_info import SparsePauliOp

H2_op = SparsePauliOp.from_list(
    [
        ("II", -1.052373245772859),
        ("IZ", 0.39793742484318045),
        ("ZI", -0.39793742484318045),
        ("ZZ", -0.01128010425623538),
        ("XX", 0.18093119978423156),
    ]
)

driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

h2_problem = driver.run()

mapper = JordanWignerMapper()

h2_reference_state = HartreeFock(
    num_spatial_orbitals=h2_problem.num_spatial_orbitals,
    num_particles=h2_problem.num_particles,
    qubit_mapper=mapper,
)

ansatz = UCC(
    num_spatial_orbitals=h2_problem.num_spatial_orbitals,
    num_particles=h2_problem.num_particles,
    qubit_mapper=mapper,
    initial_state=h2_reference_state,
    excitations=2,
)

ansatz.decompose().decompose().draw("mpl")

In [ ]:
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Sampler, Estimator
from qiskit.algorithms.state_fidelities import ComputeUncompute
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

estimator = Estimator()
sampler = Sampler()
fidelity = ComputeUncompute(sampler)

vqe = VQE(
    estimator=estimator,
    ansatz=ansatz,
    optimizer=optimizer,
    initial_point=np.zeros(ansatz.num_parameters),
)
gse = GroundStateEigensolver(qubit_mapper=mapper, solver=vqe)
result = gse.solve(h2_problem)

print(result)

Wir können VQD auch nutzen, um nach den angeregten Zuständen zu suchen

In [ ]:
from qiskit.algorithms.eigensolvers import VQD

h2_operators, _ = gse.get_qubit_operators(h2_problem)

optimizer = COBYLA()
sampler = Sampler()
estimator = Estimator()
fidelity = ComputeUncompute(sampler)

k = 3
betas = [33, 33, 33]

vqd = VQD(
    estimator=estimator,
    ansatz=ansatz,
    optimizer=optimizer,
    fidelity=fidelity,
    initial_point=np.zeros(ansatz.num_parameters),
    k=k,
    betas=betas,
)
result = vqd.compute_eigenvalues(operator=h2_operators)
vqd_values = result.optimal_values
print(vqd_values)

## Optimierung: Max-Cut

Das Maximum-Cut-Problem (Max-Cut) ist ein kombinatorisches Optimierungsproblem, bei dem die Eckpunkte eines Graphen in zwei disjunkte Mengen so aufgeteilt werden, dass die Anzahl der Kanten zwischen den beiden Mengen maximiert wird. Formeller ausgedrückt verlangt das Max-Cut-Problem bei gegebenem ungerichteten Graphen $G=(V,E)$, wobei $V$ die Menge der Eckpunkte und $E$ die Menge der Kanten ist, die Eckpunkte in zwei disjunkte Teilmengen zu unterteilen , $S$ und $T$, so dass die Anzahl der Kanten mit einem Endpunkt in $S$ und dem anderen in $T$ maximiert wird.

Wir können Max-Cut anwenden, um verschiedene Probleme zu lösen, darunter Clustering, Netzwerkdesign, Phasenübergänge usw. Wir beginnen mit der Erstellung eines Problemdiagramms:

In [ ]:
import networkx as nx

n = 4
G = nx.Graph()
G.add_nodes_from(range(n))
edge_list = [(0, 1, 1.0), (0, 2, 1.0), (0, 3, 1.0), (1, 2, 1.0), (2, 3, 1.0)]
G.add_weighted_edges_from(edge_list)

colors = ["red" for i in range(n)]


def draw_graph(G, colors):
    """Draws the graph with the chose colors"""
    layout = nx.shell_layout(G)
    nx.draw_networkx(G, node_color=colors, pos=layout)
    edge_labels = nx.get_edge_attributes(G, "weight")
    nx.draw_networkx_edge_labels(G, pos=layout, edge_labels=edge_labels)


draw_graph(G, colors)

Um bequem auf die Gewichte aller Kanten zuzugreifen, können Sie die Adjazenzmatrix verwenden. Diese Matrix kann mit [networkx.to_numpy_array()](https://networkx.org/documentation/stable/reference/generated/networkx.convert_matrix.to_numpy_array.html) abgerufen werden.

In [ ]:
w = nx.to_numpy_array(G)
print(w)

Das [`qiskit_optimization`](https://qiskit.org/documentation/optimization/index.html) Modul enthält eine Klasse namens [`Maxcut`](https://qiskit.org/documentation/optimization/stubs/qiskit_optimization.applications.Maxcut.html) , mit der Sie ein gewichtetes Max-Cut-Problem aus der Adjazenzmatrix $w$ definieren können. Aus diesem Problem können wir das entsprechende binäre Optimierungsproblem erhalten, das wir im vorherigen Abschnitt mit [`Maxcut.to_quadratic_program()`](https://qiskit.org/documentation/optimization/stubs/qiskit_optimization.applications.Maxcut.to_quadratic_program.html) abgeleitet haben.

In [ ]:
from qiskit_optimization.applications import Maxcut

max_cut = Maxcut(w)

quadratic_program = max_cut.to_quadratic_program()
print(quadratic_program.prettyprint())

Dieses Problem kann als binäres Optimierungsproblem ausgedrückt werden. Für jeden Knoten $0 \leq i &lt; n$, wobei $n$ die Anzahl der Knoten des Graphen ist (in diesem Fall $n=4$), betrachten wir die binäre Variable $x_i$. Diese Variable hat den Wert $1$, wenn der Knoten $i$ zu einer der Gruppen gehört, die wir mit $1$ bezeichnen, und $0$, wenn er sich in der anderen Gruppe befindet, die wir mit $0$ bezeichnen. Wir werden auch als $w_{ij}$ (Element $(i,j)$ der Adjazenzmatrix $w$) das Gewicht der Kante bezeichnen, die vom Knoten $i$ zum Knoten $j$ verläuft. Da der Graph ungerichtet ist, gilt $w_{ij}=w_{ji}$. Dann können wir unser Problem so formulieren, dass wir die folgende Kostenfunktion maximieren:

$$
\begin{aligned}
C(\vec{x})
& =\sum_{i,j=0}^n w_{ij} x_i(1-x_j)\\[1mm]

& = \sum_{i,j=0}^n w_{ij} x_i - \sum_{i,j=0}^n w_{ij} x_ix_j\\[1mm]

& = \sum_{i,j=0}^n w_{ij} x_i - \sum_{i=0}^n \sum_{j=0}^i 2w_{ij} x_ix_j
\end{aligned}
$$

Um dieses Problem mit einem Quantencomputer zu lösen, werden wir die Kostenfunktion als den erwarteten Wert einer Observablen ausdrücken. Die von Qiskit anerkannten Observablen bestehen jedoch ursprünglich aus Pauli-Operatoren, die die Eigenwerte $1$ und $-1$ anstelle von $0$ und $1$ haben. Deshalb werden wir folgende Variablenänderung vornehmen:

Wobei $\vec{x}=(x_0,x_1,\cdots ,x_{n-1})$. Wir können die Adjazenzmatrix $w$ verwenden, um bequem auf die Gewichte aller Kanten zuzugreifen. Dies wird verwendet, um unsere Kostenfunktion zu erhalten:

$$
z_i = 1-2x_i \rightarrow x_i = \frac{1-z_i}{2}
$$

Dies impliziert Folgendes:

$$
\begin{array}{lcl} x_i=0 & \rightarrow & z_i=1 \\ x_i=1 & \rightarrow & z_i=-1.\end{array}
$$

Die neue Kostenfunktion, die wir maximieren möchten, ist also:

$$
\begin{aligned}
C(\vec{z})
& = \sum_{i,j=0}^n w_{ij} \bigg(\frac{1-z_i}{2}\bigg)\bigg(1-\frac{1-z_j}{2}\bigg)\\[1mm]

& = \sum_{i,j=0}^n \frac{w_{ij}}{4} - \sum_{i,j=0}^n \frac{w_{ij}}{4} z_iz_j\\[1mm]

& = \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} -  \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} z_iz_j
\end{aligned}
$$

Darüber hinaus besteht die natürliche Tendenz eines Quantencomputers darin, Minima (normalerweise die niedrigste Energie) anstelle von Maxima zu finden. Anstatt also $C(\vec{z})$ zu maximieren, werden wir Folgendes minimieren:

$$
-C(\vec{z}) =  \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} z_iz_j -  \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2}
$$

Da wir nun eine Kostenfunktion zum Minimieren haben, deren Variablen die Werte $-1$ und $1$ haben können, können wir die folgende Analogie zum Pauli-$Z$ ziehen:

$$
z_i \equiv Z_i = \overbrace{I}^{n-1}\otimes ... \otimes \overbrace{Z}^{i} \otimes ... \otimes \overbrace{I}^{0}
$$

Mit anderen Worten, die Variable $z_i$ entspricht einem $Z$-Gate, das auf das Qubit $i$ wirkt. Darüber hinaus:

$$
Z_i|x_{n-1}\cdots x_0\rangle = z_i|x_{n-1}\cdots x_0\rangle \rightarrow \langle x_{n-1}\cdots x_0 |Z_i|x_{n-1}\cdots x_0\rangle = z_i
$$

Dann ist die Observable, die wir betrachten werden:

$$
\hat{H} = \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2} Z_iZ_j
$$

zu dem wir anschließend den unabhängigen Term hinzufügen müssen:

$$
\texttt{offset} = - \sum_{i=0}^n \sum_{j=0}^i \frac{w_{ij}}{2}
$$

Diese Transformation kann mit [`QuadraticProgram.to_ising()`](https://qiskit.org/documentation/optimization/stubs/qiskit_optimization.QuadraticProgram.to_ising.html) durchgeführt werden.

In [ ]:
observable, offset = quadratic_program.to_ising()
print("Offset:", offset)
print("Ising Hamiltonian:")
print(str(observable))

Wir können eine VQE-Instanz verwenden, um die optimalen Parameter wie folgt zu finden:

In [ ]:
from qiskit.primitives import Estimator
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal
import numpy as np

ansatz = TwoLocal(observable.num_qubits, "rx", reps=1)
optimizer = COBYLA()

vqe = VQE(
    estimator=Estimator(),
    ansatz=ansatz,
    optimizer=optimizer,
    initial_point=np.zeros(ansatz.num_parameters),
)

result = vqe.compute_minimum_eigenvalue(observable)

print(result)

In [ ]:
from qiskit.quantum_info import Statevector

optimal_circuit = result.optimal_circuit.bind_parameters(result.optimal_parameters)

x = max_cut.sample_most_likely(Statevector(optimal_circuit))
print("energy:", result.eigenvalue.real)
print("time:", result.optimizer_time)
print("max-cut objective:", result.eigenvalue.real + offset)
print("solution:", x)
print("solution objective:", quadratic_program.objective.evaluate(x))

# plot results
colors = ["red" if x[i] == 0 else "orange" for i in range(n)]
draw_graph(G, colors)

Mit dieser Lektion haben Sie gelernt:

- So schreiben Sie einen benutzerdefinierten Variationsalgorithmus
- So wenden Sie einen Variationsalgorithmus an, um minimale Eigenwerte zu finden
- Wie man Variationsalgorithmen nutzt, um Anwendungsanwendungsfälle zu lösen

Fahren Sie mit der letzten Lektion fort, um Ihre Bewertung abzulegen und Ihr Abzeichen zu erhalten!